In [1]:
%reload_ext autoreload
%autoreload 2

In [10]:
import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(REPO_DIR, 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

In [11]:
dataset_settings

,network_model,stain,margins,num_sample_per_class,stacks
dataset_id,,,,,
20,Inception-BN,nissl,200/500,1000,MD585
21,Inception-BN,nissl,200/500,1000,MD589
22,Inception-BN,nissl,200/500,1000,MD594


In [22]:
dataset_id = 22
dataset_properties = dataset_settings.loc[dataset_id]

num_samples_per_label = dataset_properties['num_sample_per_class']
stacks = dataset_properties['stacks'].split('/')

In [23]:
structures_to_sample = all_known_structures

negative_labels_to_sample = [s + '_negative' for s in structures_to_sample]

margins_to_sample = map(int, str(dataset_properties['margins']).split('/'))

surround_positive_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix=surr_l) 
                             for m in margins_to_sample
                             for s in structures_to_sample 
                             for surr_l in structures_to_sample
                             if surr_l != s]

surround_noclass_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix='noclass') 
                             for m in margins_to_sample
                             for s in structures_to_sample]

labels_to_sample = structures_to_sample + negative_labels_to_sample + \
surround_positive_labels_to_sample + surround_noclass_labels_to_sample + \
['noclass']

In [24]:
features, addresses, labels_found = generate_dataset(num_samples_per_label=num_samples_per_label, 
                                                     stacks=stacks,
                                                     labels_to_sample=labels_to_sample)

Read: 29.21 seconds
Cell is empty.
Cell is empty.
Cell is empty.
Sample: 32.79 seconds
Sample addresses: 62.05 seconds


('MD594', 94)
('MD594', 111)
('MD594', 128)
('MD594', 146)
('MD594', 95)
('MD594', 147)
('MD594', 96)
('MD594', 97)
('MD594', 112)
('MD594', 129)
('MD594', 98)
('MD594', 148)
('MD594', 99)
('MD594', 100)
('MD594', 149)
('MD594', 101)
('MD594', 113)
('MD594', 130)
('MD594', 102)
('MD594', 150)
('MD594', 103)
('MD594', 114)
('MD594', 151)
('MD594', 104)
('MD594', 132)
('MD594', 105)
('MD594', 152)
('MD594', 115)
('MD594', 106)
('MD594', 153)
('MD594', 133)
('MD594', 116)
('MD594', 107)
('MD594', 154)
('MD594', 117)
('MD594', 108)
('MD594', 134)
('MD594', 155)
('MD594', 118)
('MD594', 109)
('MD594', 156)
('MD594', 135)
('MD594', 119)
('MD594', 110)
('MD594', 157)
('MD594', 120)
('MD594', 163)
('MD594', 158)
('MD594', 121)
('MD594', 136)
('MD594', 122)
('MD594', 159)
('MD594', 164)
('MD594', 123)
('MD594', 137)
('MD594', 124)
('MD594', 160)
('MD594', 125)
('MD594', 165)
('MD594', 138)
('MD594', 126)
('MD594', 161)
('MD594', 139)
('MD594', 166)
('MD594', 127)
('MD594', 140)
('MD594', 162)
(

Map addresses to features: 238.05 seconds


In [25]:
# for label in sorted(features.iterkeys()):
#     print label, len(addresses[label])

In [26]:
# Save training features
features_fp = os.path.join(CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset_id, 'patch_features.hdf')
create_parent_dir_if_not_exists(features_fp)
save_hdf_v2(features, features_fp)
upload_from_ec2_to_s3(features_fp)
# train_feat_dir = create_if_not_exists(os.path.join(CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset, 'patch_features'))
# for label, feats in training_features.iteritems():
#     bp.pack_ndarray_file(feats, os.path.join(train_feat_dir, label + '.bp'))

# Save training addresses
addresses_fp = os.path.join(CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset_id, 'patch_addresses.pkl')
save_pickle(addresses, addresses_fp)
upload_from_ec2_to_s3(addresses_fp)

aws s3 cp /shared/CSHL_classifiers/datasets/dataset_22/patch_features.hdf s3://mousebrainatlas-data/CSHL_classifiers/datasets/dataset_22/patch_features.hdf


Child returned 0
1.83 seconds.


aws s3 cp /shared/CSHL_classifiers/datasets/dataset_22/patch_addresses.pkl s3://mousebrainatlas-data/CSHL_classifiers/datasets/dataset_22/patch_addresses.pkl


Child returned 0
1.22 seconds.
